<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/NDL_DocL%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88%E3%81%A8YOLOv5%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E3%83%AC%E3%82%A4%E3%82%A2%E3%82%A6%E3%83%88%E6%8A%BD%E5%87%BA%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDL-DocLデータセットとYOLOv5を用いたレイアウト抽出モデル

## セットアップ

In [ ]:
#@title

# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# koraのインストール
!pip install kora
from kora.xattr import get_id

!pip install -U --no-cache-dir gdown --pre

# 物体検出モデル（best.pt）
!gdown https://drive.google.com/uc?id=1DduqMfElGLPYWZTbrEO8F3qn6VPOZDPM

import torch
import pprint
from PIL import Image

model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

# 

import glob
import os
from tqdm import tqdm

def predict(input_dir, output_dir):

  exts = ["jpg", "JPG", "JPEG", "jpeg", "png", "PNG"]

  files = []
  for ext in exts:
    files_ = glob.glob("{}/*.{}".format(input_dir, ext))
    files.extend(files_)
  
  for file in tqdm(files):
    img = Image.open(file)
    results = model(img, size=1024)

    results.render()  # updates results.imgs with boxes and labels
    for img in results.imgs:
        img_base64 = Image.fromarray(img)
        path = output_dir + "/" + os.path.basename(file)
        os.makedirs(os.path.dirname(path), exist_ok=True)
        img_base64.save(path, format="JPEG")

  try:
    fid = get_id(output_dir)
    print("")
    print("以下のURLから出力結果を確認してください。")
    print("https://drive.google.com/drive/u/1/folders/{}".format(fid))
  except:
    pass

from google.colab import output
output.clear()

def downloadImage(url, input_dir):
  input_path = "{}/tmp.jpg".format(input_dir)
  from urllib import request
  request.urlretrieve(url, input_path)
  return input_path

## 推論の実行

### 単一の画像ファイルのURLを指定する場合

『源氏物語』(東京大学総合図書館所蔵)を利用しています。

In [ ]:
url = "https://iiif.dl.itc.u-tokyo.ac.jp/iiif/genji/TIFF/A00_6587/01/01_0004.tif/full/1024,/0/default.jpg" #@param {type:"string"}

input_dir = "/tmp"
output_dir = "/tmp"

# 画像のダウンロード
img_path = downloadImage(url, input_dir)

# 推論
predict(input_dir, output_dir)

# 結果の表示
import IPython
IPython.display.Image(img_path)

### 単一の画像ファイルをアップロードする場合

In [ ]:
#@title

input_dir = "/tmp"
output_dir = "/tmp"

# アップロードと移動
from google.colab import files
uploaded = files.upload()
input_file = next(iter(uploaded))

!mv $input_file $input_dir/tmp.jpg

# 推論
predict(input_dir, output_dir)

# 結果の表示
import IPython
IPython.display.Image(img_path)

### 複数の既にダウンロード済みの画像ファイルを対象にする場合

画像ファイルを格納した入力フォルダへのパス`input_dir`、および認識結果を格納するための出力フォルダへのパス`output_dir`を指定してください。

In [ ]:
#@title
input_dir = "/content/drive/Shareddrives/yolo/rnn/test_genji" #@param {type:"string"}
output_dir = "/content/drive/Shareddrives/yolo/projects/ndl/ml_test/genji" #@param {type:"string"}

# input_dir = "/content/drive/Shareddrives/yolo/projects/ndl/test/images" #@param {type:"string"}
# output_dir = "/content/drive/Shareddrives/yolo/projects/ndl/ml_test/test" #@param {type:"string"}

# input_dir = "/content/drive/Shareddrives/yolo/rnn/test" #@param {type:"string"}
# output_dir = "/content/drive/Shareddrives/yolo/projects/ndl/ml_test/other" #@param {type:"string"}

predict(input_dir, output_dir)